In [1]:
from selenium import webdriver
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.keys import Keys
#select works to select from an item
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
import selenium.webdriver.support.ui as ui
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup as BS
import numpy as np
import pandas as pd
import spacy
import parsedatetime
import datetime as dt
import json

In [2]:
#To simulate the delay
import random
import time
start_time = time.time()
normal_delay = random.normalvariate(2, 0.5)
time.sleep(normal_delay)    
# print("--- %.5f seconds ---" % (time.time() - start_time))

def delay(t):
    normal_delay = random.normalvariate(t, 0.5)
    time.sleep(normal_delay)

In [3]:
# Run the Chrome Driver
driver = webdriver.Chrome(executable_path=r'/chromedriver')

In [4]:
def find_str(s, char):
    index = 0
    if char in s:
        c = char[0]
        for ch in s:
            if ch == c:
                if s[index:index+len(char)] == char:
                    return index
            index += 1
    return -1

In [5]:
web_link = "https://www.amazon.com/RockBirds-Flashlights-Bright-Aluminum-Flashlight/product-reviews/B00X61AJYM"
driver.get(web_link)
delay(2)

In [6]:
driver.find_element_by_xpath("//select[@id='reviewer-type-dropdown']/option[text()='Verified purchase only']").click()
driver.find_element_by_xpath("//select[@id='sort-order-dropdown']/option[text()='Most recent']").click()

In [8]:
# find reviews sent
sent_div = driver.find_element_by_id('cm_cr-review_list')
sent_class = sent_div.find_element_by_class_name('a-section')
span = sent_class.find_element_by_xpath('//*[@id="cm_cr-review_list"]/div[1]/span[1]')
reviews = span.text

In [9]:
nlp = spacy.load('en')
doc = nlp(unicode(reviews))
review_num = doc[-2].text.encode("utf-8")
review_num = filter(str.isdigit, review_num)
review_page = int(np.ceil(int(review_num) / 10))

In [10]:
current_url = driver.current_url
index= find_str(current_url, '&pageNumber')

In [11]:
def cleanup(token, lower = True):
    if lower:
       token = token.lower()
    return token.strip()

In [12]:
def out_for(divs):
    for div in divs:
        #   find date by data-hook = review-date and //filter out review for review year < 2017
        date = div.find('span', attrs={'data-hook': 'review-date'}).get_text()
        date_nlp = nlp(unicode(date)).ents[0]
        date_year = date_nlp[-1]
        # find author name
        author = div.find('a', attrs={'data-hook': 'review-author'}).text.encode("utf-8")
        # find helpful vote statement
        vote = ""
        if div.find('span', attrs={'data-hook': 'helpful-vote-statement'}):
            vote = div.find('span', attrs={'data-hook': 'helpful-vote-statement'}).text.encode("utf-8")
        # find the title
        title = div.find('a', attrs={'data-hook': 'review-title'}).text.encode("utf-8")
        # find the rating score
        rating = div.find('span', attrs={'class': 'a-icon-alt'}).text.encode("utf-8")
        # if before 2017 break otherwise: append title,date,review
        if int(date_year.text) < 2017:
#             print('its before year 2017')
            return
        else:
            review_div = div.find('div', attrs={'class': 'a-section celwidget'})
            review = review_div.find('span', attrs={'data-hook': 'review-body'}).text.encode("utf-8")
        date_list.append(date_nlp.text.encode("utf-8"))
        author_list.append(author)
        vote_list.append(vote)
        title_list.append(title)
        review_list.append(review)
        rating_list.append(rating)

In [22]:
date_extend_list = []
author_extend_list = []
vote_extend_list = []
title_extend_list = []
review_extend_list = []
rating_extend_list = []
for i in range(1,review_page+2):
    review_page_link = current_url[0:index] + "&pageNumber=" + str(i) + "&sortBy=recent"
    driver.get(review_page_link)
    page_source = driver.page_source
    page_source = BS(page_source, "html5lib")
    review_block = page_source.find('div', attrs={'id': 'cm_cr-review_list'})
    review_divs = review_block.findAll('div', attrs={'class': 'review'})

    date_list = []
    author_list = []
    vote_list = []
    title_list = []
    review_list = []
    rating_list = []
    out_for(review_divs)
    date_extend_list.extend(date_list)
    author_extend_list.extend(author_list)
    review_extend_list.extend(review_list)
    title_extend_list.extend(title_list)
    vote_extend_list.extend(vote_list)
    rating_extend_list.extend(rating_list)

In [23]:
df = pd.DataFrame({'Title': title_extend_list,'Author': author_extend_list,'Date': date_extend_list, 'Review': review_extend_list, "How helpful": vote_extend_list, "Score": rating_extend_list})
df.to_json('reviews.json')

In [21]:
df = pd.read_json("reviews.json")
df['Review'][0]

u'Nice well made little flashlight, they work good on 1.2 volt rechargeable Ni-Mh batteries.'